## Experimento
### Biblioteca : DARTS
### Periodo:  2015 a 2020
### Horizonte de previsão: 12 meses a partir de JAN2020

### Resultados de alguns modelos para cada subestação

In [1]:
from darts import TimeSeries

In [5]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import time
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    Prophet,
    ExponentialSmoothing,
    ARIMA,
    AutoARIMA,
    RegressionModel,
    Theta,
    FFT
)
from darts.metrics import mape, mase
#from darts.utils.statistics import check_seasonality, plot_acf

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

In [6]:
def eval_model(model):
    model.fit(train)
    forecast = model.predict(len(val))
    print('model {} obtains MAPE: {:.2f}%'.format(model, mape(val, forecast)))

In [7]:
rows_list = []
for i in range(92):
    i=i+8
    dd = pd.read_csv('/home/pi/A/DARTS/dados/carga_2015_2020.csv',usecols=[0,i], header=0)
    df=dd
    df = TimeSeries.from_dataframe(df, time_col='data')
    train, val = df.split_after(pd.Timestamp("2020-01"))
    print('==================')
    print(dd.columns[1])
    print('==================')
    #eval_model(NaiveDrift())
    eval_model(NaiveSeasonal(12))
    #eval_model(FFT())
    eval_model(ARIMA())
    eval_model(AutoARIMA())
    eval_model(ExponentialSmoothing())
    eval_model(Prophet())
    eval_model(Theta())
    
    thetas = 2 - np.linspace(-10, 10, 150)
    best_mape = float('inf')
    best_theta = 0
    for theta in thetas:
        model = Theta(theta)
        model.fit(train)
        pred_theta = model.predict(len(val))
        res = mape(val, pred_theta)
        if res < best_mape:
            best_mape = res
            best_theta = theta
    best_theta_model = Theta(best_theta)
    best_theta_model.fit(train)
    pred_best_theta = best_theta_model.predict(len(val))
           
    print('Best Theta MAPE is: {:.2f}%, with theta = {}.'.format(mape(val, pred_best_theta), best_theta))
   


soma_carga
model Naive seasonal model, with K=12 obtains MAPE: 8.59%
model ARIMA(12, 1, 0) obtains MAPE: 11.61%
model Auto-ARIMA obtains MAPE: 14.63%
model Exponential smoothing obtains MAPE: 6.49%
model Prophet obtains MAPE: 6.92%
model Theta(2) obtains MAPE: 6.37%


KeyboardInterrupt: 